In [ ]:
from Framework_V3.core.config import CaseConfig, GridShape
from Framework_V3.core.geometry import FinnTubedHX
from Framework_V3.models import Frostmodell_V1
from Framework_V3.runtime.simulator import Simulator
from Framework_V3.visualisation import plot
from CoolProp.HumidAirProp import HAPropsSI

# Case Definieren

In [ ]:
T_a = 2.0
T_w = -9.1
P = 103500
RH = 0.85
v_a = 1.2

cfg = CaseConfig(
    # air data
    T_a = T_a,          # °C temperature air
    v_a = v_a,          # m/s velocity air
    p_a = P,            # Pa pressure air
    RH = RH,            # relative humidity air
    w_amb = HAPropsSI('W','T',T_a+273.15,'P',P,'R', RH),          # kg/kg water vapor moisture content
    rho_amb = 1.2,      # kg/m^3 density air
    v_kin = 1.5e-5,     # m^2/s kinematic viscosity air
    lam = 0.025,        # W/mK heat conduction coefficient air
    c_p_a = 1000,       # J/kgK heat capacity air
    D_std = 2.2e-5,     # m^2/s water vapor diffusion coefficient
    C = 900,            # 1/s empirical water vapor absorbed coefficient
    isv = 2830000,      # J/kg latent heat of desublimation

    # refrigerant data
    T_w = T_w,          # °C wall temperature

    # ice data
    rho_i = 920,        # kg/m^3 frost density (solid)
    h_sub= 334,         # kJ/kg latent heat of ablimation for water vapor

    # numerics
    t_end = 40,      # s endtime
    dt = 0.1              # s time step
)

geom = FinnTubedHX(
    n_fin = 4,           # -
    l_fin = 0.022,          # m
    d_fin = 0.0002,    # m
    fin_pitch = 0.0032,          # m
    d_tube_a = 0.00952,          # m
    tube_thickness = 0.025       # m
)

gs = GridShape(
    nx = 100,
    nr = 200,
    ntheta = 5
)

plot.plot_finned_tube_side(geom)

# Simulation laufen lassen

In [ ]:
sim = Simulator(fields=("t","s_e"))
results = sim.run(cfg, geom, gs, Frostmodell_V1)

# Resultate speichen

In [ ]:
result_file = "results_test_edge.csv"
results.to_csv(result_file)

# Resultate plotten

In [ ]:
plot_r = 50
plot_theta = 0
plot_time = int(max(results.data['t']) / cfg.dt)

plot.plot_any(kind="time vs 1D(theta)",
              x=results.data['t'],y=results.data['s_e'],
             xlabel="Zeit [s]", ylabel="Frostdicke [m]",
             title=f"Frostdicke bei θ_idx = {plot_theta}",
             theta_idx=plot_theta, marker=None)
plot.plot_any(kind="time vs 2D(r,theta)",
              x=results.data['t'],y=results.data['T_e'],
             xlabel="Zeit [s]", ylabel="Temperatur [°C]",
             title=f"Frosttemperatur bei r_idx = {plot_r} und θ_idx = {plot_theta}",
             r_idx=plot_r, theta_idx=plot_theta, marker=None)
plot.plot_any(kind="time vs 2D(r,theta)",
              x=results.data['t'],y=results.data['rho_e'],
             xlabel="Zeit [s]", ylabel="Dichte [kg/m^3]",
             title=f"Frostdichte bei r_idx = {plot_r} und θ_idx = {plot_theta}",
             r_idx=plot_r, theta_idx=plot_theta, marker=None)
plot.plot_any(kind="time vs 2D(r,theta)",
              x=results.data['t'],y=results.data['w_e'],
             xlabel="Zeit [s]", ylabel="w [kg/kg]",
             title=f"Feuchtigkeit bei r_idx = {plot_r} und θ_idx = {plot_theta}",
             r_idx=plot_r, theta_idx=plot_theta, marker=None)

In [ ]:
plot.plot_spatial_slice(
    results.data["T_e"], vary="r", t_idx=plot_time, theta_idx=plot_theta,
    title=f"T_e(r) bei t={results.data['t'][plot_time]} s, θ_idx = {plot_theta}", xlabel="r_idx", ylabel="T_e [°C]", marker=None
)
plot.plot_spatial_slice(
    results.data["rho_e"], vary="r", t_idx=plot_time, theta_idx=plot_theta,
    title=f"rho_e(r) bei t={results.data['t'][plot_time]} s, θ_idx = {plot_theta}", xlabel="r_idx", ylabel="rho_e [kg/m^3]", marker=None
)
plot.plot_spatial_slice(
    results.data["w_e"], vary="r", t_idx=plot_time, theta_idx=plot_theta,
    title=f"w_e(r) bei t={results.data['t'][plot_time]} s, θ_idx = {plot_theta}", xlabel="r_idx", ylabel="w_e [kg/kg]", marker=None
)

In [ ]:
plot.plot_spatial_slice(
    y=results.data["s_e"],     # shape: (time, theta)
    vary="theta",                 # we’re slicing along θ
    r_idx=plot_time,
    t_idx=plot_time,                 # pick the time index
    ylabel="s_e [m]",
    title=f"s_e(θ) bei t={results.data['t'][plot_time]} s" if "t" in results.data else "s_e(θ)",
    marker=None
)